In [12]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [16]:
url = "https://pre-flight.cn/WebHome/NoticeInfoTemp"
data = {'type':0, 'ids':198}
res = requests.post(url, data=data)
bs = BeautifulSoup(res.text)
title = bs.find('div', attrs = {'class' : 'gate_htitle'}).text
a_list = bs.find_all('ul', attrs = {'class': 'NoticeXZ'})[0].find_all('a')
a_data = []
for a in a_list:
    a_data.append([title, a.text, a['href']])
    
pd.DataFrame(a_data)

,0,1,2
0,2021冬春航季外籍带飞,2021冬春航季外籍带飞（开放部分）.xlsx,https://pre-flight.cn/down/198/2021冬春航季外籍带飞（开放...
1,2021冬春航季外籍带飞,2021冬春航季外籍带飞（非开放部分）.xls,https://pre-flight.cn/down/198/2021冬春航季外籍带飞（非开...


In [25]:
url = "https://pre-flight.cn/WebHome/NoticeInfoTemp"
data = {'type':0, 'ids':190}
res = requests.post(url, data=data)
bs = BeautifulSoup(res.text)
ul = bs.find('ul', attrs = {'class': 'NoticeXZ'})
ul == None

True

In [27]:
import threading
a_data = []
def get_link(i):
    link_data = {}
    data = {'type':0, 'ids':i}
    res = requests.post(url, data=data)
    bs = BeautifulSoup(res.text)
    link_data['title'] = bs.find('div', attrs = {'class' : 'gate_htitle'}).text
    try:
        ul = bs.find('ul', attrs = {'class': 'NoticeXZ'})
        if ul == None:
            print("No link is found in id {}".format(i))
            return
        else:
            a_list = ul.find_all('a')
            for a in a_list:
                link_data['description'] = a.text
                link_data['link'] = a['href']
    except Exception as e:
        print('error with {}'.format(i), str(e))
        
    a_data.append(link_data)

thread_list = []
for i in range(198):
    thread = threading.Thread(target=get_link, args=(i,))
    thread_list.append(thread)
    thread.start()

for thread in thread_list:
    thread.join()

flight_dl = pd.DataFrame(a_data)
flight_dl

No link is found in id 52
No link is found in id 54
No link is found in id 51
No link is found in id 120
No link is found in id 124
No link is found in id 53
No link is found in id 65
No link is found in id 50
No link is found in id 121
No link is found in id 166
No link is found in id 190
No link is found in id 82
No link is found in id 167
No link is found in id 141
No link is found in id 122
No link is found in id 95
No link is found in id 155
No link is found in id 182
No link is found in id 158
No link is found in id 191
No link is found in id 168
No link is found in id 160
No link is found in id 183
No link is found in id 172


,title,description,link
0,,,
1,,,
2,,,
3,,,
4,,,
...,...,...,...
169,关于发布国内航空公司2016冬春航季部分定期飞行计划的通知,2016冬春国内航空公司港澳台航班正班计划(EXCEL格式),https://pre-flight.cn/down/A21042016冬春国内航空公司港澳...
170,,,
171,2020冬春航季航空公司所聘外籍和台湾地区飞行员加入机组执行航班任务（开放部分二次增补）,2020冬春航季航空公司所聘外籍和台湾地区飞行员加入机组执行航班任务（开放部分二次增补）.xls,https://pre-flight.cn/down/180/2020冬春航季航空公司所聘外...
172,2020冬春航季航空公司所聘外籍和台湾地区飞行员加入机组执行航班任务,2020冬春航季航空公司所聘外籍和台湾地区飞行员加入机组执行航班任务（开放部分增补）.xlsx,https://pre-flight.cn/down/180/2020冬春航季航空公司所聘外...


In [45]:
from contextlib import closing
import csv
import os
def download(title, url):
    path = r'.\download'
    if not os.path.exists(path):
        try:
            os.mkdir(path)
        except:
            print('Path existed. Ignore>>>')
    suffix = url.split(".")[-1]
    r = requests.get(url, stream=True)
    f = open(r'{}\{}.{}'.format(path, title, suffix), "wb")
    for chunk in r.iter_content(chunk_size=512):
        if chunk:
            f.write(chunk)
    
    f.close()

In [47]:
%%time
flight_dl = flight_dl.dropna(subset=['title', 'link'])
thread_list = []
for index in flight_dl.index:
    title = flight_dl.loc[index, 'title'].replace('/','／')
    url = flight_dl.loc[index, 'link']

    if url != '':
        thread = threading.Thread(target=download, args=(title, url, ))
        thread_list.append(thread)
        thread.start()
        # download(title, url)

for thread in thread_list:
    thread.join()
    

Wall time: 21.9 s
